In [23]:
!pip install scikit-image

  Using cached scikit_image-0.20.0-cp310-cp310-win_amd64.whl (23.7 MB)
     ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
      --------------------------------------- 0.1/3.4 MB 1.7 MB/s eta 0:00:03
     -- ------------------------------------- 0.2/3.4 MB 2.4 MB/s eta 0:00:02
     ---- ----------------------------------- 0.4/3.4 MB 2.8 MB/s eta 0:00:02
     ------ --------------------------------- 0.6/3.4 MB 3.3 MB/s eta 0:00:01
     ------------ --------------------------- 1.1/3.4 MB 4.9 MB/s eta 0:00:01
     ----------------------- ---------------- 2.0/3.4 MB 7.1 MB/s eta 0:00:01
     ---------------------------------------  3.4/3.4 MB 10.8 MB/s eta 0:00:01
     ---------------------------------------- 3.4/3.4 MB 9.4 MB/s eta 0:00:00
  Using cached tifffile-2023.3.21-py3-none-any.whl (218 kB)
  Using cached PyWavelets-1.4.1-cp310-cp310-win_amd64.whl (4.2 MB)
  Using cached lazy_loader-0.2-py3-none-any.whl (8.6 kB)


In [2]:
pip list 

Package                      Version
---------------------------- ------------
absl-py                      1.4.0
anyio                        3.5.0
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
asttokens                    2.0.5
attrs                        22.1.0
Babel                        2.11.0
backcall                     0.2.0
beautifulsoup4               4.11.1
bleach                       4.1.0
brotlipy                     0.7.0
certifi                      2022.12.7
cffi                         1.15.1
charset-normalizer           2.1.1
colorama                     0.4.6
comm                         0.1.2
comtypes                     1.1.14
contourpy                    1.0.7
cryptography                 39.0.1
cycler                       0.11.0
debugpy                      1.5.1
decorator                    5.1.1
defusedxml                   0.7.1
distlib                      0.3.6
entrypoints                  0.4
executing                    0.8.3


In [15]:
# cd ./parking-spot-detection

C:\Users\Admin\Desktop\CV\parking-spot-detection


In [16]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
from skimage.transform import resize

In [45]:
data_list = os.listdir("./data")
data_paths = []
for data in data_list:
    data_path = os.path.join("./data", data)
    data_paths.append(data_path)
data_paths

['./data\\mask_1920_1080.png',
 './data\\mask_crop.png',
 './data\\parking_1920_1080.mp4',
 './data\\parking_1920_1080_loop.mp4',
 './data\\parking_crop.mp4',
 './data\\parking_crop_loop.mp4']

In [ ]:
EMPTY = True
NOT_EMPTY = False

MODEL = pickle.load(open("car_spot_detection.p", "rb"))

def empty_or_not(spot_bgr):

    flat_data = []

    img_resized = resize(spot_bgr, (15, 15, 3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)

    y_output = MODEL.predict(flat_data)

    if y_output == 0:
        return EMPTY
    else:
        return NOT_EMPTY

In [ ]:
def get_parking_spots_bboxes(connected_components):
    (totalLabels, label_ids, values, centroid) = connected_components

    slots = []
    coef = 1
    for i in range(1, totalLabels):

        # Now extract the coordinate points
        x1 = int(values[i, cv2.CC_STAT_LEFT] * coef)
        y1 = int(values[i, cv2.CC_STAT_TOP] * coef)
        w = int(values[i, cv2.CC_STAT_WIDTH] * coef)
        h = int(values[i, cv2.CC_STAT_HEIGHT] * coef)

        slots.append([x1, y1, w, h])

    return slots

In [136]:
def calc_diff(img1, img2):
    return np.abs(np.mean(img1) - np.mean(img2))

In [152]:
mask = cv2.imread(data_paths[0], 0)

cap = cv2.VideoCapture(data_paths[2])

connected_video = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)

spots = get_parking_spots_bboxes(connected_video)
spots_status = [None for _ in spots]

diffs = [None for _ in spots]
previous_frame = None

frame_num = 0
step = 50

while True:
    ret, frame = cap.read()
    
    car_counter = 0
    empty_spot_counter = 0
    
    if frame_num%step == 0 and previous_frame is not None:
        for idx, spot in enumerate(spots):
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1+h, x1:x1+w]
            
            diffs[idx] = calc_diff(spot_crop, previous_frame[y1:y1+h, x1:x1+w])
    
    if frame_num%step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j]/np.amax(diffs) > 0.4]
        
        for idx in arr_:
            spot = spots[idx]
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1+h, x1:x1+w]
            spot_status = empty_or_not(spot_crop)
            spots_status[idx] = spot_status
            
           
            
    if frame_num%step == 0:
        previous_frame = frame.copy()
    
    
    for idx, spot in enumerate(spots):  
        spot_status = spots_status[idx]
        x1, y1, w, h = spots[idx]
        
        if spot_status==True:
            cv2.rectangle(frame, (x1,y1), (x1+w,y1+h), (0,0,255), 3)
            empty_spot_counter += 1
        else:
            cv2.rectangle(frame, (x1,y1), (x1+w,y1+h), (255,0,0), 3)
            car_counter += 1
    
    total_spot = car_counter + empty_spot_counter
    
    frame[0:70, 10: 1350] = [0,0,0]
    cv2.putText(frame, f"Car:{car_counter}/{total_spot} - Available spot:{empty_spot_counter}/{total_spot}",
                (25,50), cv2.FONT_HERSHEY_PLAIN, 4, (0,255,0), 4)
    
#         cv2.putText(frame, f"{idx}",(x1,y1+20), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)
        
     
    cv2.namedWindow("Video", cv2.WINDOW_NORMAL)   
    cv2.imshow("Video", frame)
    k = cv2.waitKey(10)
    if k == ord("q"):
        break
    frame_num += 1
    
cap.release()
cv2.destroyAllWindows()

In [143]:
cap.release()
cv2.destroyAllWindows()